In [107]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [108]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from tensorflow.keras.models import Model


# 이미지 및 라벨링 데이터 디렉토리 설정
image_dir = "/content/drive/MyDrive/ColabNotebooks/archive/www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/normal"
label_dir = "/content/drive/MyDrive/ColabNotebooks/archive/www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/val2"

In [109]:
# 이미지 디렉토리에서 파일 목록 가져오기
image_files = os.listdir(image_dir)

# 라벨링 디렉토리에서 파일 목록 가져오기
label_files = os.listdir(label_dir)

print("이미지 파일 목록:")
print(image_files)

print("라벨링 파일 목록:")
print(label_files)

이미지 파일 목록:
['Screenshot (310).png', 'Screenshot (309).png', 'Screenshot (311).png', 'Screenshot (312).png', 'Screenshot (322).png', 'Screenshot (313).png', 'Screenshot (318).png', 'Screenshot (321).png', 'Screenshot (319).png', 'Screenshot (314).png', 'Screenshot (317).png', 'Screenshot (324).png', 'Screenshot (323).png', 'Screenshot (354).png', 'Screenshot (353).png', 'Screenshot (325).png', 'Screenshot (326).png', 'Screenshot (357).png', 'Screenshot (356).png', 'Screenshot (361).png', 'Screenshot (359).png', 'Screenshot (355).png', 'Screenshot (358).png', 'Screenshot (366).png', 'Screenshot (368).png', 'Screenshot (367).png', 'Screenshot (364).png', 'Screenshot (362).png', 'Screenshot (363).png', 'Screenshot (372).png', 'Screenshot (374).png', 'Screenshot (369).png', 'Screenshot (370).png', 'Screenshot (371).png', 'Screenshot (373).png', 'Screenshot (380).png', 'Screenshot (377).png', 'Screenshot (375).png', 'Screenshot (376).png', 'Screenshot (378).png', 'Screenshot (379).png', 'Scr

In [105]:
import tensorflow as tf

# 데이터 증강
def augment1(image,label): # 밝기 조절
    delta = tf.random.uniform(shape=[], minval=-max_delta, maxval=max_delta)
    # 이미지의 밝기를 조절
    image = tf.image.adjust_brightness(image, delta)
    return image, label

def augment2(image, label):
    image = tf.image.random_flip_left_right(image) # 랜덤하게 좌우를 반전합니다.
    return image,label


In [106]:
os.getcwd()


'/content'

In [102]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from tensorflow.keras.models import Model

# 이미지 및 라벨링 데이터 디렉토리 설정
image_dir = "/content/drive/MyDrive/ColabNotebooks/archive/www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/normal"
label_dir = "/content/drive/MyDrive/ColabNotebooks/archive/www.acmeai.tech ODataset 1 - Motorcycle Night Ride Dataset/val2"

# 이미지 크기 설정
input_shape = (256, 256, 3)

# U-Net 모델 정의
def unet_model(input_shape):
    inputs = Input(input_shape)

    # Encoding
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)

    # Decoding
    up5 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(up5)
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(conv7)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv7)

    model = Model(inputs, outputs)
    return model

# U-Net 모델 생성
model = unet_model(input_shape)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 이미지 및 라벨 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_path, label_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = image / 255.0  # 이미지 정규화 (0~1 범위)

    label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
    label = cv2.resize(label, (input_shape[0], input_shape[1]))
    label = label / 255.0  # 라벨링 데이터 정규화 (0~1 범위)
    label = np.expand_dims(label, axis=-1)

    return image, label

# 데이터셋 준비
image_files = os.listdir(image_dir)
label_files = os.listdir(label_dir)

X_train = []
y_train = []

for image_file in image_files:
    if image_file in label_files:
        image_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, image_file)
        image, label = load_and_preprocess_data(image_path, label_path)
        X_train.append(image)
        y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# 모델 훈련
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)


ValueError: ignored